In [65]:
import gym
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt  
import numpy as np
import random
from collections import deque
from collections import namedtuple


In [66]:
class exp_replay():
    
    
    
    def __init__(self, mem_size,  b_size):
        self.mem = deque(maxlen=mem_size)
        self.b_size = b_size
        self.exp_tuple = namedtuple("Exp", field_names=["states","actions", "reward", "next_states", "done"])
        
        
    def append_exp(self, states, actions, rewards, next_states, done):
        states = torch.from_numpy(states).type(torch.float)
        actions = torch.from_numpy(actions).type(torch.float)
        rewards = torch.from_numpy(rewards).type(torch.float)
        next_states = torch.from_numpy(next_states).type(torch.float)
        done = torch.from_numpy(done).type(torch.float)
        exp = self.exp_tuple(states, actions, rewards, next_states, done)
        self.mem.append(exp)
        
    def get_batch(self):
        
        batch = random.sample(self.mem, self.b_size)
        
        states_b = torch.stack([i.states for i in batch])
        rewards_b = torch.stack([i.rewards for i in batch])
        next_states_b = torch.stack([i.next_states for i in batch])
        done_b = torch.stack([i.done for i in done])
        actions_b = torch.stack([i.actions for i in batch])
        return states_b, actions_b, rewards_b, next_states_b, done_b
        
        

In [67]:
class Actor(nn.Module):
    
    def __init__(self, input_dim, output_dim, hidden_nodes):
        super(Policy, self).__init__()
        self.input_dim = input_dim
        self.output_dimput_dim = output_dim
        self.hn = hidden_nodes
        
        self.fc1 = nn.Linear(self.input_dim, self.hn)
        self.fc2 = nn.Linear(self.hn, self.hn)
        self.fc3 = nn.Linear(self.hn, self.output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = Categorical(F.softmax(self.fc3(x, dim=1))) # This computes a distribution from which we can sample and take log of for the gradient decent
        return x 
    
class Critic(nn.Module):
    
    def __init__(self, input_dim, hidden_nodes):
        self.input_dim = input_dim
        self.hn = hidden_nodes
        
        self.fc1 = nn.Linear(self.input_dim, self.hn)
        self.fc2 = nn.Linear(self.hn , self.hn)
        self.fc3 = nn.Linear(self.hn, 1) # Outputs a Q value of the state
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
def train(env, epochs, actor, critic):
    
    actor_optimizer = optim.Adam(actor.parameters())
    critic_optimizer = optim.Adam(critic.parameters())
    
    for i in range(epochs):
        obs = env.reset()
        
        while not done:
            obs = torch.from_numpy(obs).type(torch.float)
            # Do we need the exploration exploitation?
            action_prob  = actor(obs)
            action = action_prob.sample()
            
        
    
    